## Setup

In [1]:
# import packages and modules
import os
import csv
from datetime import datetime
import pyodbc
import arcpy
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
import pickle
import pandas as pd
from functools import reduce

arcpy.overwriteOutput = True

# set local variables
workspace =  "C:\\GIS\PROJECT\\ThresholdEvaluation\\ThresholdEvaluation.gdb"
sdeBase   = "C:\\GIS\\DB_CONNECT\\Vector.sde"
sdeCollect= "C:/GIS/DB_CONNECT/Collection.sde"

# sde datasets
parcels    = os.path.join(sdeBase,"sde.SDE.Parcels\sde.SDE.Parcel_Master")
soils      = os.path.join(sdeBase, "sde.SDE.Soils\sde.SDE.land_capability_NRCS2007") 
localplan  = os.path.join(sdeBase, "sde.SDE.Planning\sde.SDE.LocalPlan")
towncenter = os.path.join(sdeBase, "sde.SDE.Planning\sde.SDE.TownCenter")
lcv        = os.path.join(sdeBase, "sde.SDE.Soils\sde.SDE.Land_Capability_Verification")
ipes       = os.path.join(sdeCollect, "sde_collection.SDE.Parcel\sde_collection.SDE.Parcel_LTinfo_IPES")
                          
# local impervious datasets
newImp = os.path.join(workspace, "Impervious_2018")
oldImp = os.path.join(workspace, "Impervious_2010")
changeImp = os.path.join(workspace, "Impervious_Change_2010_2019")                        
# sde impervious
impervious2010Old = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_2010")
impervious2010 = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_2010_Cleaned")                                                      
impervious2018 = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_2019")
imperviousChng = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_Change_2010_2019")
# local impervious layers


# output tables
outTable2010Old = "ImperviousOverlayAnalysis_2010_Original.csv"
outTable2010 = "ImperviousOverlayAnalysis_2010.csv"
outTable2018 = "ImperviousOverlayAnalysis_2018.csv"
outTable2018LCVHybrid = "ImperviousOverlayAnalysis_2018_LCVHybrid.csv"
outTableChng = "ImperviousOverlayAnalysis_Change.csv"
outLocation  = "C:\\GIS\PROJECT\\ThresholdEvaluation\\"

# Create Land Capability Hybrid Layer

In [2]:
lcvslct = arcpy.SelectLayerByAttribute_management(lcv, 'NEW_SELECTION', 
                                        "LCV_IPES IN ('1A', '1B', '1C', '2', '3', '4', '5', '6', '7')")

lcvHybrid = os.path.join(workspace, "Land_Capability_Hybrid")

In [3]:
# delete hybrid layer if it exists
if arcpy.Exists(lcvHybrid):
    arcpy.Delete_management(lcvHybrid)

# union all lc layers together to create hybrid land capability layer
arcpy.Union_analysis([[soils,3], [lcvslct, 1], [ipes, 2]], 
                     lcvHybrid, 
                     "ALL", 
                     0.1)


<Result 'C:\\GIS\\PROJECT\\ThresholdEvaluation\\ThresholdEvaluation.gdb\\Land_Capability_Hybrid'>

In [5]:
# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(lcvHybrid, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

Calculating Acres...


In [6]:
sdf = pd.DataFrame.spatial.from_featureclass(lcvHybrid)
sdf.head()

,OBJECTID,FID_land_capability_NRCS2007,MUSYM,MUKEY,sde_SDE_land_capability_NRCS2007_area,Map_Symbol,Land_Capab,Percent_Al,Bailey_Co,Allowed_ac,...,APN_1,PPNO_1,JURISDICTION_1,dAddDate,SQFT,CAP,LCV_IPES,Soil_Type,IPES_Score,SHAPE
0,1,1,7222,1652092,29.836887,7222.0,6,26.0,0.3,8.950801,...,,0.0,,1899-12-30,0.0,,,,,"{""rings"": [[[751161.5832926724, 4348756.977501..."
1,2,3,9405,1652041,232.233094,9405.0,4,18.0,0.2,46.444619,...,,0.0,,1899-12-30,0.0,,,,,"{""rings"": [[[764547.1237890339, 4357046.003537..."
2,3,4,7156,1652093,10.642096,7156.0,4,18.0,0.2,2.128355,...,,0.0,,1899-12-30,0.0,,,,,"{""rings"": [[[758459.4885912314, 4348798.389694..."
3,4,5,7151,1652177,10.029084,7151.0,6,27.0,0.3,3.008659,...,,0.0,,1899-12-30,0.0,,,,,"{""rings"": [[[751310.6695681289, 4348327.372572..."
4,5,7,7222,1652092,30.579129,7222.0,6,26.0,0.3,9.173550,...,,0.0,,1899-12-30,0.0,,,,,"{""rings"": [[[753049.905681801, 4348557.9678345..."


In [7]:
# iterating the columns 
for col in sdf.columns: 
    print(col) 

OBJECTID
FID_land_capability_NRCS2007
MUSYM
MUKEY
sde_SDE_land_capability_NRCS2007_area
Map_Symbol
Land_Capab
Percent_Al
Bailey_Co
Allowed_ac
GISAcre
FID_Parcel_LTinfo_IPES
ScoreSheetUrl
APN
Status
ParcelNickname
IPESScore
IPESScoreType
BaseAllowableCoveragePercent
IPESTotalAllowableCoverageSqFt
ParcelHasDOAC
HistoricOrImportedIpesScore
CalculationDate
FieldEvaluationDate
RelativeErosionHazardScore
RunoffPotentialScore
AccessScore
UtilityInSEZScore
ConditionOfWatershedScore
AbilityToRevegetateScore
WaterQualityImprovementsScore
ProximityToLakeScore
LimitedIncentivePoints
TotalParcelArea
IPESBuildingSiteArea
SEZLandArea
SEZSetbackArea
InternalNotes
PublicNotes
PPNO
PARCEL_ACRES
PARCEL_SQFT
JURISDICTION
FID_Land_Capability_Verification
APN_1
PPNO_1
JURISDICTION_1
dAddDate
SQFT
CAP
LCV_IPES
Soil_Type
IPES_Score
SHAPE


In [8]:
sdf.set_index('OBJECTID')

,FID_land_capability_NRCS2007,MUSYM,MUKEY,sde_SDE_land_capability_NRCS2007_area,Map_Symbol,Land_Capab,Percent_Al,Bailey_Co,Allowed_ac,GISAcre,...,APN_1,PPNO_1,JURISDICTION_1,dAddDate,SQFT,CAP,LCV_IPES,Soil_Type,IPES_Score,SHAPE
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,1,7222,1652092,29.836887,7222.0,6,26.0,0.3,8.950801,29.814506,...,,0.0,,1899-12-30 00:00:00.000000,0.000000,,,,,"{""rings"": [[[751161.5832926724, 4348756.977501..."
2,3,9405,1652041,232.233094,9405.0,4,18.0,0.2,46.444619,232.018547,...,,0.0,,1899-12-30 00:00:00.000000,0.000000,,,,,"{""rings"": [[[764547.1237890339, 4357046.003537..."
3,4,7156,1652093,10.642096,7156.0,4,18.0,0.2,2.128355,10.633123,...,,0.0,,1899-12-30 00:00:00.000000,0.000000,,,,,"{""rings"": [[[758459.4885912314, 4348798.389694..."
4,5,7151,1652177,10.029084,7151.0,6,27.0,0.3,3.008659,10.021509,...,,0.0,,1899-12-30 00:00:00.000000,0.000000,,,,,"{""rings"": [[[751310.6695681289, 4348327.372572..."
5,7,7222,1652092,30.579129,7222.0,6,26.0,0.3,9.173550,26.608665,...,,0.0,,1899-12-30 00:00:00.000000,0.000000,,,,,"{""rings"": [[[753049.905681801, 4348557.9678345..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31788,3799,7221,1652146,37.361270,7221.0,6,26.0,0.3,11.208018,0.008730,...,111-070-034,111070034.0,PL,2017-05-10 15:24:16.000001,380.590200,20061023STD,6,TaD,None,"{""rings"": [[[754902.9896767316, 4349271.710217..."
31789,3799,7221,1652146,37.361270,7221.0,6,26.0,0.3,11.208018,0.008730,...,111-070-046,111070046.0,PL,2017-05-10 15:24:16.000001,41341.701444,20061023STD,6,TaD,None,"{""rings"": [[[754902.9896767316, 4349271.710217..."
31790,3799,7221,1652146,37.361270,7221.0,6,26.0,0.3,11.208018,0.063686,...,111-070-019,111070019.0,PL,2017-05-10 15:24:16.000001,2776.373211,20061023STD,6,TaD,None,"{""rings"": [[[754911.0758754006, 4349268.564717..."


In [9]:
sdf.LCV_IPES.unique()

array(['', '1A', '6', '5', '1B', '3', '4', '2', '1C', '7'], dtype=object)

In [10]:
# Using DataFrame.drop
df = sdf[['OBJECTID','FID_land_capability_NRCS2007', 'Land_Capab', 'GISAcre', 
          'FID_Parcel_LTinfo_IPES', 'Status', 'IPESScore','IPESScoreType',
          'RelativeErosionHazardScore','RunoffPotentialScore',
          'FID_Land_Capability_Verification', 'LCV_IPES',
#           'OWNERSHIP_TYPE', 'TRPA_LANDUSE_DESCRIPTION', 'JURISDICTION_1',
#           'PLAN_NAME', 'ZONING_ID', 'ZONING_DESCRIPTION','TOWN_CENTER', 'LOCATION_TO_TOWNCENTER',
          'GISAcre','SHAPE']]

## IPES to Land Capability Translation

In [11]:
 
df.loc[((df.FID_Parcel_LTinfo_IPES != -1), "IPES_Translated_Score")] = df['RelativeErosionHazardScore'] + df['RunoffPotentialScore']

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [12]:
df

,OBJECTID,FID_land_capability_NRCS2007,Land_Capab,GISAcre,FID_Parcel_LTinfo_IPES,Status,IPESScore,IPESScoreType,RelativeErosionHazardScore,RunoffPotentialScore,FID_Land_Capability_Verification,LCV_IPES,GISAcre,SHAPE,IPES_Translated_Score
0,1,1,6,29.814506,-1,,0.0,,0.0,0.0,-1,,29.814506,"{""rings"": [[[751161.5832926724, 4348756.977501...",NaN
1,2,3,4,232.018547,-1,,0.0,,0.0,0.0,-1,,232.018547,"{""rings"": [[[764547.1237890339, 4357046.003537...",NaN
2,3,4,4,10.633123,-1,,0.0,,0.0,0.0,-1,,10.633123,"{""rings"": [[[758459.4885912314, 4348798.389694...",NaN
3,4,5,6,10.021509,-1,,0.0,,0.0,0.0,-1,,10.021509,"{""rings"": [[[751310.6695681289, 4348327.372572...",NaN
4,5,7,6,26.608665,-1,,0.0,,0.0,0.0,-1,,26.608665,"{""rings"": [[[753049.905681801, 4348557.9678345...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31787,31788,3799,6,0.008730,-1,,0.0,,0.0,0.0,8181,6,0.008730,"{""rings"": [[[754902.9896767316, 4349271.710217...",NaN
31788,31789,3799,6,0.008730,-1,,0.0,,0.0,0.0,19314,6,0.008730,"{""rings"": [[[754902.9896767316, 4349271.710217...",NaN
31789,31790,3799,6,0.063686,-1,,0.0,,0.0,0.0,8073,6,0.063686,"{""rings"": [[[754911.0758754006, 4349268.564717...",NaN
31790,31791,3799,6,0.063686,-1,,0.0,,0.0,0.0,8182,6,0.063686,"{""rings"": [[[754911.0758754006, 4349268.564717...",NaN


In [13]:
df.rename(columns={'Land_Capab':'LandCapability_NRCS',
                   'LCV_IPES':'LandCapability_Verified',
                    'JURISDICTION_1':'JURISDICTION'}, 
                 inplace=True)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
df.loc[(df.IPES_Translated_Score == 0), 'Land_Capability_Modified'] = '1B'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '1A'), 'Land_Capability_Modified'] = '1A'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '1C'), 'Land_Capability_Modified'] = '1C'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '2'), 'Land_Capability_Modified'] = '2'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '1B'), 'Land_Capability_Modified'] = '1B'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '3'), 'Land_Capability_Modified'] = '3'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '4'), 'Land_Capability_Modified'] = '4'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '5'), 'Land_Capability_Modified'] = '5'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '6'), 'Land_Capability_Modified'] = '6'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '7'), 'Land_Capability_Modified'] = '7'
df.loc[(df.IPES_Translated_Score > 269) & (df.IPES_Translated_Score < 380), 'Land_Capability_Modified'] = '3'
df.loc[(df.IPES_Translated_Score > 379) & (df.IPES_Translated_Score < 415), 'Land_Capability_Modified'] = '4'
df.loc[(df.IPES_Translated_Score > 414) & (df.IPES_Translated_Score < 511), 'Land_Capability_Modified'] = '5'
df.loc[(df.IPES_Translated_Score > 510) & (df.IPES_Translated_Score < 600), 'Land_Capability_Modified'] = '6'
df.loc[(df.IPES_Translated_Score > 599), 'Land_Capability_Modified'] = '7'

df.loc[(df.FID_Parcel_LTinfo_IPES == -1) & (df.FID_Land_Capability_Verification.notna()), 'Land_Capability_Modified'] = df.LandCapability_Verified

df.loc[(df.FID_Parcel_LTinfo_IPES == -1) & (df.FID_Land_Capability_Verification == -1), 'Land_Capability_Modified'] = df.LandCapability_NRCS

df.loc[(df.Land_Capability_Modified.isnull()), 'Land_Capability_Modified'] = df.LandCapability_NRCS

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
df.Land_Capability_Modified.unique()


array(['6', '4', '5', '3', ' ', '2', '7', '1C', '1A', '1B'], dtype=object)

In [16]:
df['Land_Capability_Modified'].fillna('NA', inplace=True)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [17]:
df.set_index('OBJECTID')

,FID_land_capability_NRCS2007,LandCapability_NRCS,GISAcre,FID_Parcel_LTinfo_IPES,Status,IPESScore,IPESScoreType,RelativeErosionHazardScore,RunoffPotentialScore,FID_Land_Capability_Verification,LandCapability_Verified,GISAcre,SHAPE,IPES_Translated_Score,Land_Capability_Modified
OBJECTID,,,,,,,,,,,,,,,
1,1,6,29.814506,-1,,0.0,,0.0,0.0,-1,,29.814506,"{""rings"": [[[751161.5832926724, 4348756.977501...",NaN,6
2,3,4,232.018547,-1,,0.0,,0.0,0.0,-1,,232.018547,"{""rings"": [[[764547.1237890339, 4357046.003537...",NaN,4
3,4,4,10.633123,-1,,0.0,,0.0,0.0,-1,,10.633123,"{""rings"": [[[758459.4885912314, 4348798.389694...",NaN,4
4,5,6,10.021509,-1,,0.0,,0.0,0.0,-1,,10.021509,"{""rings"": [[[751310.6695681289, 4348327.372572...",NaN,6
5,7,6,26.608665,-1,,0.0,,0.0,0.0,-1,,26.608665,"{""rings"": [[[753049.905681801, 4348557.9678345...",NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31788,3799,6,0.008730,-1,,0.0,,0.0,0.0,8181,6,0.008730,"{""rings"": [[[754902.9896767316, 4349271.710217...",NaN,6
31789,3799,6,0.008730,-1,,0.0,,0.0,0.0,19314,6,0.008730,"{""rings"": [[[754902.9896767316, 4349271.710217...",NaN,6
31790,3799,6,0.063686,-1,,0.0,,0.0,0.0,8073,6,0.063686,"{""rings"": [[[754911.0758754006, 4349268.564717...",NaN,6


In [18]:
# export cleaned up lcv layer with modified field for land capability
df.spatial.to_featureclass(lcvHybrid)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


'C:\\GIS\\PROJECT\\ThresholdEvaluation\\ThresholdEvaluation.gdb\\Land_Capability_Hybrid'

In [19]:
df.GISAcre.sum()

328166.0683107383

In [20]:
np.round(pd.pivot_table(df, index="Land_Capability_Modified",values=["GISAcre"],aggfunc=np.sum),0)

,GISAcre
Land_Capability_Modified,
,126229.0
1A,23966.0
1B,11411.0
1C,53708.0
2,23235.0
3,17805.0
4,31450.0
5,12170.0
6,23191.0


# Land Capability/Impervious Overlay Analysis

### New 2010 Impervious Analysis

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2010")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: id_NRCS_Impervious_LocalPlan_TownCenter_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, impervious2010, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2010)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2010))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2010)

# success?
print("Table {0} created".format(outTable2010))

### 2018 Impervious Analysis

In [ ]:
lcvHybrid

In [21]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2018")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(lcvHybrid, impervious2018, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2018)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2018))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2018)

# success?
print("Table {0} created".format(outTable2018LCVHybrid))

Identifying Soils with Impervious Surfaces...
Identifying Soils with Impervious Surfaces & Local Plan layers...
Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...
Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...
Calculating Acres...
Deleting Old Table...
Exporting New Table...


ExecuteError: ERROR 001156: Failed on input OID 1, could not write value '' to output field IPESScoreType
Failed to execute (TableToTable).


In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2018")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, impervious2018, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2018)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2018))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2018)

# success?
print("Table {0} created".format(outTable2018))

## Overlay Overlay

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_Change")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(lcvHybrid, imperviousChng, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTableChng)):
    arcpy.Delete_management(os.path.join(outLocation,outTableChng))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTableChng)

# success?
print("Table {0} created".format(outTableChng))

In [4]:
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTableChng)):
    arcpy.Delete_management(os.path.join(outLocation,outTableChng))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTableChng)

# success?
print("Table {0} created".format(outTableChng))

Deleting Old Table...
Exporting New Table...


ExecuteError: ERROR 001156: Failed on input OID 1, could not write value '' to output field IPESScoreType
Failed to execute (TableToTable).


## Old 2010 Overlay

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2010Old")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, impervious2010Old, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2010Old)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2010Old))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2010Old)

# success?
print("Table {0} created".format(outTable2010Old))

# Change Analysis

In [ ]:
workspace   = r"C:\GIS\PROJECT\ImperviousSurface\ImperviousSurface.gdb"
sdfImp2010  =  pd.DataFrame.spatial.from_featureclass(os.path.join(workspace, "Impervious_2010_Cleaned"))
sdfImp2018  =  pd.DataFrame.spatial.from_featureclass(os.path.join(workspace, "Impervious_2018_2019"))
sdfImpChange =  pd.DataFrame.spatial.from_featureclass(os.path.join(workspace, "Impervious_2018_2019_ChangeAnalysis"))

In [ ]:
sdfImpChange.info()

In [ ]:
sdfImpChange.General_Status.unique()

In [ ]:
sdfImp2018

In [ ]:
sdf